# Micro Session 9: Dashboarding with Python Assignment - Answer Key

### Exercise 1 - Interactive Line Charts
Load up the Apple data set and create two widgets to allow the user to pick start/end for a line chart of the closing prices.
1. Import data (AAPL.csv). Make sure to parse the dates and set them as the index.
2. Create and test the output of the line chart with hard coded variables for the start/end dates.
3. Create a function for the output you created in step 2, where the inputs are the start and end dates as strings.
4. Create 2 widgets for picking the start and end dates.
5. Connect the widgets to the function.

In [ ]:
#Import Packages
import ipywidgets as widgets
import pandas as pd
import plotly.express as px
from ipywidgets import interact, interact_manual
from IPython.display import display

In [ ]:
#Step 1 - import data
aapl = pd.read_csv('StockData/AAPL.csv', parse_dates=['Date'], index_col=['Date'])

In [ ]:
#Step 2 - create and test the outputs you want with python code
start = '2016-05-05'
end = '2017-05-05'
filtered = aapl.loc[start:end]
fig = px.line(filtered, y='Close')
fig.show()

In [ ]:
#Step 3 - create a function for the outputs
def createLineChart(start, end):
    filtered = aapl.loc[start:end]
    fig = px.line(filtered, y='Close')
    fig.show()
    display(filtered)

In [ ]:
# createLineChart("2017-01-01",'2018-01-05')

In [ ]:
#Step 4 - create the widgets and test them
startPick = widgets.DatePicker(description='Start:', value=aapl.index.min())
endPick = widgets.DatePicker(description='End:', value=aapl.index.max())
display(endPick) #Test output of widget

In [ ]:
#Step 5 - connect the widgets to the function
output1 = widgets.interactive_output(createLineChart, {"start":startPick, "end":endPick})

# display(startPick, endPick)
display(widgets.HBox([startPick, endPick]))
display(output1)

### Exercise 2 - Interactive Dashboards

In this exercise you will practice creating interactive elements to dynamically update charts. The dashboard will showcase the interactivity of plotly express charts by slicing the Financing Deals data set by different categories.
1. Load the financing deals data set from the Data Manipulation Worksheet.xlsx file in ExData.	
    - Use the Financing Table Clean tab.
2. Create a function that grabs two inputs and plots a Plotly graph.
    - One of the inputs should be the header name of the field to be plotted on the x-axis and the other the header of the field to be shown in the legend.
3. Create two widgets, one dropdown to select the data to be plotted on the x-axis, and one dropdown for the data to be plotted in the legend of the graph.
4. Preview the dashboard using Voila.

Challenge: Show the dropdowns side by side instead of next to each other and also add a textbox to search and filter the dataframe for a specific lead underwriter. (Hint: use interactive_output and widget.HBox or .VBox)

In [ ]:
#Import Packages
import ipywidgets as widgets
import pandas as pd
import plotly.express as px
from ipywidgets import interact, interact_manual
from IPython.display import display

#Import Data
finData = pd.read_excel("ExData/Data Manipulation Worksheet.xlsx",
                        sheet_name="Financing Table Clean")
cols = finData.columns

def dynamicPlot(xData, legendData):
    fig = px.bar(finData, x=xData, y='SIZE', color=legendData,
                 title="Total Deal Value by {} and {}".format(xData,legendData))
    fig.show()

xDrop = widgets.Dropdown(options=cols, value='INDUSTRY', description='Pick x-Axis:')
legendDrop = widgets.Dropdown(options=cols, value='TYPE', description='Pick legend:')

dash = interact(dynamicPlot, xData=xDrop, legendData=legendDrop)

In [ ]:
#Challenge
def dynamicPlot(xData, legendData, bankName):
    filterDF = finData[finData['LEAD UNDERWRITER'].str.contains(bankName)]
    fig = px.bar(filterDF, x=xData, y='SIZE', color=legendData,
                 title="Total Deal Value by {} and {}".format(xData,legendData))
    fig.show()
    
def showDF(bankName, showDF):
    if showDF:
        filterDF = finData[finData['LEAD UNDERWRITER'].str.contains(bankName)]
        display(filterDF)

#widgets
xDrop = widgets.Dropdown(options=cols, value='INDUSTRY', description='Pick x-Axis:')
legendDrop = widgets.Dropdown(options=cols, value='TYPE', description='Pick legend:')
showDFCheck = widgets.Checkbox(value=False, description='Show List of Deals')
bankInput = widgets.Text(value="", description='Search for lead underwriter')


out1 = widgets.interactive_output(dynamicPlot,
                                  {"xData":xDrop, "legendData":legendDrop, "bankName":bankInput})
out2 = widgets.interactive_output(showDF,
                                  {"showDF":showDFCheck, "bankName":bankInput})

dash_widgets = widgets.HBox([xDrop, legendDrop, showDFCheck])
widgets.VBox([bankInput, dash_widgets, out1, out2])

## Advanced Question
### Exercise 3 - Analysis of S&P 500 Constituents

In this exercise you will practice many of the concepts learned in training, including exploring, cleaning up and merging data sets with Pandas, creating calculated fields and interactive visualizations and dashboards.

1. Import packages below.
2. Load up the following two data sets from the ExData folder:
- sp500_SectorData.csv - this contains sector information of each of the S&P500 constitutents
- sp500_Cos_ValMetrics.csv - this contains valuation metrics of the S&P500 companies

3. Merge the two dataframes into one table.
4. Box Plots - create two different boxplot graphs with Seaborn:
    - One showing distribution of Betas by GICS Sector
    - One showing EV/Revenue by GICS Sector
    
5. You will notice that one of the graphs is skewed due to outliers. Explore the data to understand which companies represent those outliers. (Hint: perform a filter on the EV/Revenue column). What do you notice is common about the companies?

6. Recreate the second Box Plot without the outliers found.

7. Apply a "cleaning up" function on the Enterprise Value column to remove the B's and T's and convert the billions and trillions values into USD$ millions.

8. Create a dashboard that allows the user to pick from two dropdowns the following:
- GICS Sector to plot
- Type of graph to show

The options for graphs should be:
- a Plotly Sunburst chart: https://plotly.com/python/sunburst-charts/
- a Plotly Tree Map chart: https://plotly.com/python/treemaps/
- a Plotly Bar chart with GICS Sub-Industry as the x-axis: https://plotly.com/python/bar-charts/

All three charts should show EV in USD millions as the value. For the Sunburst and Tree Map, the "path" should be: GICS Sub-Industry --> Symbol.

Hint: Use an if statement in the function tied to the interactive widgets.

Run the dashboard through Voila preview to interact with the widgets without seeing the codes.

In [ ]:
#Import packages
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from IPython.display import display
from ipywidgets import interact, interactive

In [ ]:
#Import Data
sectors = pd.read_csv("ExData/sp500_SectorData.csv")
valMetrics = pd.read_csv("ExData/sp500_Cos_ValMetrics.csv")


In [ ]:
sectors.info()
valMetrics.info()

In [ ]:
#sectors
#sectors['GICS Sector'].value_counts()

In [ ]:
#Merge Data
stockData = sectors.merge(valMetrics, how='outer', on='Symbol')
        # left_on = ['Date'], right_on=['Date']
# stockData.info()
stockData[stockData['Enterprise Value'].isnull()]

In [ ]:
#BoxPlot
sns.boxplot(y='GICS Sector', x='Beta', data=stockData)
sns.set(rc={"figure.figsize":(10, 10)})
plt.show()

In [ ]:
sns.boxplot(y='GICS Sector', x='Enterprise Value/Revenue', data=stockData)
sns.set(rc={"figure.figsize":(10, 10)})
plt.show()

In [ ]:
#Outliers in Consumer Discretionary
stockData[stockData['Enterprise Value/Revenue']>50]

In [ ]:
#Remove outliers - by filtering out large EV/Sales
filterDF = stockData[stockData['Enterprise Value/Revenue']<50]
sns.boxplot(y='GICS Sector', x='Enterprise Value/Revenue', data=filterDF)
sns.set(rc={"figure.figsize":(10, 10)})
plt.show()

In [ ]:
#Clean up the Enterprise value column by removing the B and T and keeping all values as millions USD
def convertEV(ev):
    if type(ev) == str:
        if ev.find('B') >= 0:
            ev = float(ev.replace('B','')) * 1000
        elif ev.find('T') >= 0:
            ev = float(ev.replace('T','')) * 1000000
        else:
            ev = float(ev)
    return ev
stockData['EV mm'] = stockData['Enterprise Value'].apply(convertEV)
stockData['Market Cap mm'] = stockData['Market Cap (intraday)'].apply(convertEV)
stockData

#stockData['Enterprise Value'].str.strip("B").str.strip("T").astype(float)

In [ ]:
stockData.info()

In [ ]:
#Interactive Dashboard
def sectorPlot(sector, graphtype):
    filterData = stockData[stockData['GICS Sector'] == sector] 
    print("Total EV of ${:,.0f}".format(filterData['EV mm'].sum()))
    if graphtype == 'Sunburst':
        fig = px.sunburst(filterData, path=['GICS Sub-Industry', 'Symbol'], values='EV mm')
    elif graphtype == 'Tree Map':
        fig = px.treemap(filterData, path=['GICS Sub-Industry', 'Symbol'], values='EV mm')
    elif graphtype == 'Bar':
        fig = px.bar(filterData, x='GICS Sub-Industry', y='EV mm', color='Symbol')
    fig.update_layout(
        title={
            'text': "EV ($mm) by Sector: " + sector,
            'y':0.99,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'})
    fig.show()

    
sectorPick = interact(sectorPlot, sector=stockData['GICS Sector'].unique(),
                      graphtype=['Sunburst','Tree Map','Bar'])